<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-проекта" data-toc-modified-id="Описание-проекта-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание проекта</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Импорт-библиотек-и-данных" data-toc-modified-id="Импорт-библиотек-и-данных-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Импорт библиотек и данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Предобработка данных</a></span></li><li><span><a href="#Обработка-текстового-столбца" data-toc-modified-id="Обработка-текстового-столбца-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обработка текстового столбца</a></span></li><li><span><a href="#Разбиение" data-toc-modified-id="Разбиение-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Разбиение</a></span></li><li><span><a href="#Векторизация-текстов" data-toc-modified-id="Векторизация-текстов-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Векторизация текстов</a></span></li></ul></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#Dummy-Classifier" data-toc-modified-id="Dummy-Classifier-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Dummy Classifier</a></span></li><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Logistic Regression</a></span></li><li><span><a href="#Decision-Tree" data-toc-modified-id="Decision-Tree-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Decision Tree</a></span></li><li><span><a href="#Random-Forest" data-toc-modified-id="Random-Forest-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Random Forest</a></span></li></ul></li><li><span><a href="#Финальное-тестирование" data-toc-modified-id="Финальное-тестирование-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Финальное тестирование</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

## Описание проекта

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

**План работы**

1. Импортировать исходные данные.
1. Проверить данные на необходимость проведения предобработки.
1. Провести предобработку текстовых данных.
1. Векторизировать тексты.
1. Обучить модели:
    1. Логистической регрессии.
    1. Дерева решений.
    1. Случайного леса.
1. Выбрать наилучшую модель.
1. Провести финальное тестирование.

## Подготовка данных

### Импорт библиотек и данных

In [1]:
# загрузим необходимые для работы библиотеки
import pandas as pd
import numpy as np
from tqdm.contrib.itertools import product

import nltk
from nltk import word_tokenize
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from pymystem3 import Mystem
import re

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE = 12345

In [2]:
# импортируем таблицу с данными
try:
    data = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)
except:
    # здесь была ссылка на загрузку данных с сервера Яндекса

### Предобработка данных

In [3]:
# посмотрим на исходную таблицу
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


Пропусков в данных нет, формат корректный.

In [5]:
# посмотрим на распределение 0 и 1 в целевой переменной
data['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

Число нетоксичных комментариев составляет примерно 10% от всех комментариев.

In [6]:
# проверим наличие дубликатов в данных
data.duplicated().sum()

0

Дубликаты отсутствуют.

### Обработка текстового столбца

In [7]:
# добавим функцию для очистки текста от неинформативных знаков
def clear_text(text):
    text = re.sub('\n', ' ', text)
    text = re.sub(r"[^a-zA-Z']", ' ', text)
    text = text.split()
    text = " ".join(text)
    return text

In [8]:
def join_text(text):
    text = " ".join(text)
    return text

In [9]:
# добавим функцию для приведения текста к нижнему регистру
def lower_string(text):
    text = text.lower()
    return text

In [10]:
stopwords = nltk.corpus.stopwords.words('english')

In [11]:
# добавим функцию для удаления стоп-слов
def remove_stopwords(text):
    text = [token.strip() for token in text if token not in stopwords]
    return text

In [12]:
# добавим функцию для выделения части речи (вспомогательную в лемматизации)
def get_pos(token):
    
    tag = nltk.pos_tag([token])[0][1][0].upper()
    tag_dict = {'J': wordnet.ADJ,
                'N': wordnet.NOUN,
                'V': wordnet.VERB,
                'R': wordnet.ADV}
    
    return tag_dict.get(tag, wordnet.NOUN)

In [13]:
# добавим функцию для лемматизации текста
def list_lemmatize(text):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    text = [lemmatizer.lemmatize(token, get_pos(token)) for token in text]
    return text

In [14]:
# очистим тексты от неинформативных знаков
data['preprocessed'] = data['text'].apply(clear_text)

In [15]:
# посмотрим, что получилось
data.head()

,text,toxic,preprocessed
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D'aww He matches this background colour I'm se...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I'm really not trying to edit war It's...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can't make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


In [16]:
# приведем все тексты к нижнему регистру
data['preprocessed'] = data['preprocessed'].apply(lower_string)

In [17]:
# проверка
data.head()

,text,toxic,preprocessed
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d'aww he matches this background colour i'm se...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i'm really not trying to edit war it's...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can't make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


In [18]:
# токенизируем тексты
data['preprocessed'] = data['preprocessed'].apply(word_tokenize)

In [19]:
# проверка
data.head()

,text,toxic,preprocessed
0,Explanation\nWhy the edits made under my usern...,0,"[explanation, why, the, edits, made, under, my..."
1,D'aww! He matches this background colour I'm s...,0,"[d'aww, he, matches, this, background, colour,..."
2,"Hey man, I'm really not trying to edit war. It...",0,"[hey, man, i, 'm, really, not, trying, to, edi..."
3,"""\nMore\nI can't make any real suggestions on ...",0,"[more, i, ca, n't, make, any, real, suggestion..."
4,"You, sir, are my hero. Any chance you remember...",0,"[you, sir, are, my, hero, any, chance, you, re..."


In [20]:
# удалим стоп-слова
data['preprocessed'] = data['preprocessed'].apply(remove_stopwords)

In [21]:
data.head()

,text,toxic,preprocessed
0,Explanation\nWhy the edits made under my usern...,0,"[explanation, edits, made, username, hardcore,..."
1,D'aww! He matches this background colour I'm s...,0,"[d'aww, matches, background, colour, 'm, seemi..."
2,"Hey man, I'm really not trying to edit war. It...",0,"[hey, man, 'm, really, trying, edit, war, 's, ..."
3,"""\nMore\nI can't make any real suggestions on ...",0,"[ca, n't, make, real, suggestions, improvement..."
4,"You, sir, are my hero. Any chance you remember...",0,"[sir, hero, chance, remember, page, 's]"


In [22]:
%%time
data['preprocessed'] = data['preprocessed'].apply(list_lemmatize)

CPU times: user 15min 36s, sys: 2min 28s, total: 18min 4s
Wall time: 18min 7s


In [23]:
data.head()

,text,toxic,preprocessed
0,Explanation\nWhy the edits made under my usern...,0,"[explanation, edits, make, username, hardcore,..."
1,D'aww! He matches this background colour I'm s...,0,"[d'aww, match, background, colour, 'm, seeming..."
2,"Hey man, I'm really not trying to edit war. It...",0,"[hey, man, 'm, really, try, edit, war, 's, guy..."
3,"""\nMore\nI can't make any real suggestions on ...",0,"[ca, n't, make, real, suggestion, improvement,..."
4,"You, sir, are my hero. Any chance you remember...",0,"[sir, hero, chance, remember, page, 's]"


In [24]:
data['preprocessed'] = data['preprocessed'].apply(join_text)

In [25]:
data.head()

,text,toxic,preprocessed
0,Explanation\nWhy the edits made under my usern...,0,explanation edits make username hardcore metal...
1,D'aww! He matches this background colour I'm s...,0,d'aww match background colour 'm seemingly stu...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man 'm really try edit war 's guy constant...
3,"""\nMore\nI can't make any real suggestions on ...",0,ca n't make real suggestion improvement wonder...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page 's


Таким образом, мы провели токенизацию, удаление стоп-слов и лемматизацию текста.

### Разбиение

In [26]:
train, valid, test = np.split(data, [int(.9*len(data)), int(.95*len(data))])

### Векторизация текстов

In [27]:
count_tf_idf = TfidfVectorizer()
tf_idf_train = count_tf_idf.fit_transform(train['preprocessed'])

In [28]:
# признаки обучающей выборки
features_train = tf_idf_train
# таргет обучающей выборки
target_train = train['toxic']

In [29]:
# признаки валидационной выборки
features_valid = count_tf_idf.transform(valid['preprocessed']) 
# таргет валидационной выборки
target_valid = valid['toxic']

In [30]:
# признаки валидационной выборки
features_test = count_tf_idf.transform(test['preprocessed']) 
# таргет валидационной выборки
target_test = test['toxic']

In [31]:
# обучающая выборка
print(features_train.shape)
print(target_train.shape)

(143362, 143190)
(143362,)


In [32]:
# валидационная выборка
print(features_valid.shape)
print(target_valid.shape)

(7965, 143190)
(7965,)


In [33]:
# тестовая выборка
print(features_test.shape)
print(target_test.shape)

(7965, 143190)
(7965,)


**Вывод:** <br/>

В данном разделе мы импортировали исходные данные, проверили их на корректность. Нам удалось токенизировать и лемматизировать тексты, избавиться от стоп-слов. Следом за этим мы провели векторизацию текстов и разбили данные на обучающую, валидационную и тестовую выборки в соотношении 90:5:5. Далее можно приступать к обучению.

## Обучение моделей

### Dummy Classifier

In [34]:
predict_train = np.ones(shape=features_train.shape[0])
predict_valid = np.ones(shape=features_valid.shape[0])
predict_test = np.ones(shape=features_test.shape[0])

print(f'F1-мера на обучающей выборке: {f1_score(predict_train, target_train)}')
print(f'F1-мера на валидационной выборке: {f1_score(predict_valid, target_valid)}')
print(f'F1-мера на тестовой выборке: {f1_score(predict_test, target_test)}')

F1-мера на обучающей выборке: 0.1842913503004819
F1-мера на валидационной выборке: 0.1870945715261181
F1-мера на тестовой выборке: 0.18523581681476417


С предсказаниями дамми-классификатора будем сравнивать модели для оценки адекватности предсказаний.

### Logistic Regression

In [35]:
%%time
best_f1 = 0
best_c = 1

for c in range(1,30):
    
    model = LogisticRegression(random_state=RANDOM_STATE, 
                               solver='liblinear', 
                               class_weight='balanced',
                               C=c)
    model.fit(features_train, target_train)
    if best_f1 < f1_score(model.predict(features_valid), target_valid):
        best_f1 = f1_score(model.predict(features_valid), target_valid)
        best_c = c
    
print(f'Лучший C = {best_c}, лучшая средняя f1-мера = {best_f1}')

Лучший C = 9, лучшая средняя f1-мера = 0.7733039161610589
CPU times: user 7min 10s, sys: 4.4 s, total: 7min 14s
Wall time: 1min 51s


Наибольшее значение получается при C=9. Получившееся значение F1-меры=0.77 превышает требуемый порог в 0.75, что говорит о том, что модель подходит для оценки токсичных комментариев.

### Decision Tree

In [36]:
best_f1 = 0
best_max_depth = 1
best_min_samples_leaf = 1
best_model = None

for depth, leaf in product(range(1, 11), range(1, 6)):
    
    model = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=depth, min_samples_leaf=leaf)
    model.fit(features_train, target_train)
    
    if best_f1 < f1_score(model.predict(features_valid), target_valid):
        best_f1 = f1_score(model.predict(features_valid), target_valid)
        best_max_depth = depth
        best_min_samples_leaf = leaf
        best_model = model
        
print(f'Лучшая модель: {best_model}, F1-мера = {best_f1}')

  0%|          | 0/50 [00:00<?, ?it/s]

Лучшая модель: DecisionTreeClassifier(max_depth=10, min_samples_leaf=3, random_state=12345), F1-мера = 0.6220472440944882


Модель не подходит, т.к. метрика качества ниже бейзлайна.

### Random Forest

In [37]:
best_f1 = 0
best_max_depth = 1
best_min_samples_leaf = 1
best_n_estimators = 1
best_model = None

for depth, leaf, est in product(range(1, 11), range(1, 6), range(1, 51, 10)):
    
    model = RandomForestClassifier(random_state=RANDOM_STATE, 
                                   max_depth=depth, 
                                   min_samples_leaf=leaf, 
                                   n_estimators=est)
    model.fit(features_train, target_train)
    
    if best_f1 < f1_score(model.predict(features_valid), target_valid):
        best_f1 = f1_score(model.predict(features_valid), target_valid)
        best_max_depth = depth
        best_min_samples_leaf = leaf
        best_n_estimators = est
        best_model = model
        
print(f'Лучшая модель: {best_model}, F1-мера = {best_f1}')

  0%|          | 0/250 [00:00<?, ?it/s]

Лучшая модель: RandomForestClassifier(max_depth=9, min_samples_leaf=2, n_estimators=1,
                       random_state=12345), F1-мера = 0.03488372093023256


F1-мера вышла крайне низкая, случайный лес так же не подходит для оценки в данном случае.

**Вывод:** <br/>
В данном разделе мы обучили несколько моделей на векторизованных текстовых данных. Наилучшее значение F1-меры было получено в модели логистической регрессии. Деревянные модели хуже всего справились с поставленной задачей и показали крайне низкие значения метрики качества. На модели логита можно проводить финальное тестирование.

## Финальное тестирование

In [38]:
model = LogisticRegression(random_state=RANDOM_STATE, 
                           solver='liblinear', 
                           class_weight='balanced', 
                           C=best_c)
model.fit(features_train, target_train)
f1_score(model.predict(features_test), target_test)

0.7582227351413733

**Вывод:** <br/> 
Проведя проверку качества модели на тестовой выборке, мы убедились в том, что она дает хороший результат и может быть использована на практике. Метрика также лучше метрики дамми-классификатора, что говорит об адекватности модели.

## Общий вывод

В данном проекте перед нами стояла задача классификации комментариев на токсичные и нетоксичные на основе текста, содержащегося в них. Была предпринята попытка решения данной задачи через TF-IDF.

На первом этапе мы ознакомились с исходными данными и провели предобработку столбца с текстами комментариев. В рамках предобработки были выполненые: 
- очистка от неинформативных знаков, 
- приведение к нижнему регистру, 
- токенизация, 
- удаление стоп-слов,
- лемматизация.

Далее мы векторизовали тексты на основе TF-IDF. Полученные векторы использовались в обучении в качестве объясняющей переменной.

Было построено 3 модели: 
1. Логистическая регрессия.
1. Дерево решений (с подбором гиперпараметров).
1. Случайный лес (с подбором гиперпараметров).

Наилучшее значение F1-меры на валидационной выборке дала модель логистической регрессии - `0.77`. Эта модель была выбрана в качестве решения - на тестовой выборке F1-мера составила `0.76`. Бейзлайн в `0.75` был пройден.